# Lab. 1-1 Text2SQL Basic (Athena & Amazon S3)

#### 이 실습에서는 Text2SQL을 활용해서 S3에 저장된 데이터에 Athena 쿼리로 접근하는 방법을 실습합니다.

![Intro](../images/text2sql/athena-s3-1.png)


#### Amazon S3에 저장된 로그나 데이터마트에 자연어로 데이터를 조회하려는 경우, Text2SQL 및 Athena를 사용할 수 있습니다.

#### 여기서는 샘플 쿼리와 스키마 정보를 하나의 컨텍스트로 제공합니다. 데이터 접근 방식이 단순하고 사용자의 질문이 정형화 되어있는 경우, 이렇듯 가장 간단하게 Text2SQL을 시도할 수 있습니다.

## Step 0: 라이브러리 설치 및 Athena 연결

In [ ]:
!python -m ensurepip --upgrade
!pip install "sqlalchemy" --quiet
!pip install "boto3>=1.34.116"  --quiet
!pip install "jinja2" --quiet
!pip install "botocore" --quiet
!pip install "pandas" --quiet
!pip install "PyAthena" --quiet
!pip install "faiss-cpu" --quiet

In [ ]:
import json
import boto3
import sys

sys.path.append('../')
from libs.din_sql import din_sql_lib as dsl

In [ ]:
ATHENA_CATALOG_NAME = '' # check https://us-west-2.console.aws.amazon.com/cloudformation/home?region=us-west-2#/stacks
ATHENA_RESULTS_S3_LOCATION = '' # check https://us-west-2.console.aws.amazon.com/cloudformation/home?region=us-west-2#/stacks
DB_NAME = "tpcds1"

In [ ]:
from libs.din_sql import din_sql_lib as dsl

model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'

din_sql = dsl.DIN_SQL(bedrock_model_id=model_id)

In [ ]:
din_sql.athena_connect(catalog_name=ATHENA_CATALOG_NAME, 
               db_name=DB_NAME, 
               s3_prefix=ATHENA_RESULTS_S3_LOCATION)

## Step 1: 프롬프트 구성

In [ ]:
return_sql= din_sql.find_fields(db_name=DB_NAME)
print(return_sql)

In [ ]:
import os
import os
import jinja2 as j

question = "Which customer spent the most money in the web store?"

instructions_tag_start = '<instructions>'
instructions_tag_end = '</instructions>'
example_tag_start = '<example>'
example_tag_end = '</example>'
sql_tag_start = '```sql'
sql_tag_end = '```'

template_dir = "../libs/din_sql/prompt_templates"

template_file = os.path.join(template_dir, 'basic_prompt.txt.jinja')
if not os.path.isfile(template_file):
    raise FileNotFoundError(f"Template file '{template_file}' not found")

JINJA_ENV = j.Environment(
    loader=j.FileSystemLoader(template_dir),
    autoescape=j.select_autoescape(
        enabled_extensions=('jinja'),
        default_for_string=True,
    )
)

easy_prompt = JINJA_ENV.get_template('basic_prompt.txt.jinja')
prompt = easy_prompt.render(
    instruction_tag_start=instructions_tag_start,
    instruction_tag_end=instructions_tag_end,
    fields=return_sql,
    example_tag_start=example_tag_start,
    example_tag_end=example_tag_end,
    test_sample_text=question,
    sql_tag_start=sql_tag_start,
    sql_tag_end=sql_tag_end
)
print(prompt)

## Step 2: LLM을 사용해 쿼리 생성

In [ ]:
import json
import boto3

bedrock_client = boto3.client(service_name='bedrock-runtime')

def llm_generation(prompt, stop_sequences=[], word_in_mouth=None):
    user_message =  {"role": "user", "content": prompt}
    messages = [user_message]
    if word_in_mouth:
        messages.append({
            "role": "assistant",
            "content": word_in_mouth,
        })
    response = bedrock_client.invoke_model(
        modelId=model_id,
        body=json.dumps({
            "anthropic_version": "bedrock-2023-05-31",
            "messages": messages,
            "temperature": 0,
            "max_tokens": 8000,
            "stop_sequences": stop_sequences,
            })
    )
    response_dict = json.loads(response.get('body').read().decode("utf-8"))
    results = response_dict["content"][0]["text"]
    return results

sql_qry = llm_generation(prompt)

In [ ]:
SQL = sql_qry.split('```sql')[1].split('```')[0].strip()
print(f"{SQL}")

## Step 3: Athena 쿼리 실행

In [ ]:
import pandas as pd
result_set = din_sql.query(SQL)
pd.DataFrame(result_set)

## Step 4: 쿼리 결과를 활용해 답변 - 생략

#### 여기에서는 하나의 LLM 호출로 쿼리를 생성했으나, 사용자의 질문 유형에 따라 복잡한 쿼리를 생성해야 하는 경우 잘 동작하지 않을 것입니다.
#### 이후 실습에서는 이를 개선하는 방법에 대해 테스트합니다.